In [1]:
from moexalgo import Market, Ticker, session, CandlePeriod
from datetime import datetime, timedelta
import pandas as pd
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import time
from colorama import Fore, Style
import requests


load_dotenv()


class Config:
    Login = os.getenv('LOGIN')  # Адрес электронной почты, указанный при регистрации на сайте moex.com
    Password = os.getenv('PASSWORD')  # Пароль от учетной записи на сайте moex.com

print('Authorisation:', session.authorize(Config.Login, Config.Password))
stocks = Market('stocks')
sber = Ticker('SBER')

print(stocks)
print(sber)

Authorisation: True
Market('shares/TQBR')
Stock('SBER/TQBR')


In [3]:
def required_date(day:int):
    return (datetime.now() - timedelta(days=day)).date()

first_part = sber.tradestats(start=required_date(365), end=required_date(250))
second_part = sber.tradestats(start=required_date(249), end=required_date(120))
third_part = sber.tradestats(start=required_date(120), end=required_date(0))

year_trades = pd.concat([first_part, second_part, third_part])
year_trades

,ticker,tradedate,tradetime,pr_open,pr_high,pr_low,pr_close,pr_std,vol,val,...,vol_b,vol_s,disb,pr_vwap_b,pr_vwap_s,systime,sec_pr_open,sec_pr_high,sec_pr_low,sec_pr_close
0,SBER,2023-07-14,10:05:00,246.30,246.30,245.83,246.08,0.0001,40390,99375771.0,...,19333,21057,-0.04,246.07,246.02,2023-10-26 19:12:56,None,None,None,None
1,SBER,2023-07-14,10:10:00,246.04,246.09,245.37,245.79,0.0000,66770,164019893.0,...,16006,50764,-0.52,245.63,245.65,2023-10-26 19:12:56,None,None,None,None
2,SBER,2023-07-14,10:15:00,245.80,245.83,245.50,245.53,0.0000,26006,63864748.0,...,10797,15209,-0.17,245.59,245.57,2023-10-26 19:12:57,None,None,None,None
3,SBER,2023-07-14,10:20:00,245.55,245.90,245.50,245.86,0.0001,27655,67934232.0,...,17251,10404,0.25,245.68,245.60,2023-10-26 19:12:58,None,None,None,None
4,SBER,2023-07-14,10:25:00,245.85,245.87,245.70,245.78,0.0000,10566,25970325.0,...,6511,4055,0.23,245.80,245.77,2023-10-26 19:12:58,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,SBER,2024-06-26,16:25:00,324.40,324.45,323.91,324.18,0.0000,21177,68666300.0,...,9642,11535,-0.09,324.32,324.19,2024-06-26 16:25:11,None,None,None,None
9996,SBER,2024-06-26,16:30:00,324.17,324.25,324.13,324.23,0.0000,7149,23178134.0,...,4821,2328,0.35,324.23,324.19,2024-06-26 16:30:11,None,None,None,None
9997,SBER,2024-06-26,16:35:00,324.22,324.24,323.85,323.90,0.0000,16714,54148900.0,...,5221,11493,-0.38,324.01,323.96,2024-06-26 16:35:11,None,None,None,None
9998,SBER,2024-06-26,16:40:00,323.92,323.98,323.90,323.94,0.0000,9896,32058476.0,...,7781,2115,0.57,323.96,323.94,2024-06-26 16:40:11,None,None,None,None


In [4]:
#пример анализа данных торговых объемов

# Загрузка данных о сделках
trades = year_trades

# Группировка сделок по размеру
trades['trade_size'] = trades['volume'] * trades['price']
small_trades = trades[trades['trade_size'] < 10000]
large_trades = trades[trades['trade_size'] >= 10000]

# Визуализация активности крупных и мелких игроков
plt.figure(figsize=(12, 6))

# График активности мелких игроков
plt.subplot(2, 1, 1)
small_trades.groupby(pd.to_datetime(small_trades['timestamp']).dt.date)['trade_size'].sum().plot(kind='bar')
plt.title('Активность мелких игроков')
plt.xlabel('Дата')
plt.ylabel('Суммарный объем мелких сделок')

# График активности крупных игроков 
plt.subplot(2, 1, 2)
large_trades.groupby(pd.to_datetime(large_trades['timestamp']).dt.date)['trade_size'].sum().plot(kind='bar')
plt.title('Активность крупных игроков')
plt.xlabel('Дата')
plt.ylabel('Суммарный объем крупных сделок')

plt.tight_layout()
plt.show()

# Анализ активности крупных игроков
print('Активность крупных игроков:')
print(large_trades.groupby(pd.to_datetime(large_trades['timestamp']).dt.date)['trade_size'].sum().sort_values(ascending=False).head(5))

# Анализ активности мелких игроков 
print('\nАктивность мелких игроков:')
print(small_trades.groupby(pd.to_datetime(small_trades['timestamp']).dt.date)['trade_size'].sum().sort_values(ascending=False).head(5))


Matplotlib is building the font cache; this may take a moment.


KeyboardInterrupt: 

In [ ]:
import os
import pandas as pd

# Укажите путь к директории, где находятся ваши файлы
directory = '/Users/alexeyeliseev/Downloads/ticks/SBER'

# Укажите имя итогового файла
output_file = 'combined_file.csv'

# Создадим пустой DataFrame для объединения всех файлов
combined_df = pd.DataFrame()

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory, filename))
        combined_df = pd.concat([combined_df, df])

# Сохраняем объединенный DataFrame в итоговый файл
combined_df.to_csv(output_file, index=False)

print(f"Все CSV-файлы объединены в {output_file}")